In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
import gc

In [ ]:
X_train = pd.read_csv('../input/2nd-place-lightgbm-solution/X_train.csv')
y_train = pd.read_csv('../input/2nd-place-lightgbm-solution/y_train.csv')
X_test = pd.read_csv('../input/2nd-place-lightgbm-solution/X_test.csv')


In [ ]:
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini



In [ ]:
from sklearn.model_selection import KFold
import joblib

fold_score = []
kf = KFold(n_splits=10)

for fold, (train_idx, test_idx) in enumerate(kf.split(X_train)):

    gc.collect()
    clf = LogisticRegression(solver='liblinear')
    clf.fit(X_train.loc[train_idx], y_train.iloc[:,1][train_idx])
    fold_score.append(eval_gini(y_train.iloc[:,1][test_idx], clf.predict_proba(X_train.loc[test_idx])[:,1]))
    joblib.dump(clf, 'model_fold_' +str(fold)+ '.txt') 


In [ ]:
print('Cross Validation score = %1.5f' % np.mean(fold_score))

In [ ]:
result = []


for fold in range(10):
    gc.collect()
    clf = joblib.load('model_fold_' +str(fold)+ '.txt')
    result.append(clf.predict_proba(X_test)[:,1])

In [ ]:
submission_df = pd.read_csv('../input/porto-seguro-safe-driver-prediction/sample_submission.csv')

In [ ]:
submission_df['target'] = np.mean(result,axis=0)

In [ ]:
submission_df.to_csv('submission.csv', index=False)